# Load the pretrained model

Archive Images (on Server!)

In [ ]:
tar -czf /home/alex/all_imgs_07_01_24.tar.gz

Download archive from server and extract it

In [1]:
!scp alex@foodsnapai.com:/home/alex/delta_imgs.tar.gz /home/alex/delta_imgs.tar.gz
!tar -xzf /home/alex/delta_imgs.tar.gz -C /home/alex/allImgs_extracted/

delta_imgs.tar.gz                             100%  871MB  92.8MB/s   00:09    
3392d64b-75eefce8689d7102_IMG_20230802_203236.jpg
5e332ce9-155350565f93ada7_IMG_20240107_155443_HDR.jpg
99256d4c-f9c2d351859657e9_IMG_20240107_161850_HDR.jpg
063a06c6-d04ff2f1599c141c_IMG_20240107_161941_HDR.jpg
c3e72227-b7bee88fca68a47f_IMG_20240107_155226_HDR.jpg
09776733-839cc2913cb02ca3_EAA45F81-CAA1-41EF-BA7A-FDF3014AAD4D.jpeg
25dbc8bd-76da42e358d6a6e6_IMG_20240107_154152_HDR.jpg
05885819-da097dd25bd1765e_1000057480.jpg
a6e3be64-9d27ec8516a7c88d_1000057234.jpg
05184788-cb96bc02bbbfb32e_IMG_20240107_155408_HDR.jpg
246c9f39-967d5bef481efb1c_20231110_150444.jpg
78bcf2cc-1cd8087115a0ca03_1000057237.jpg
84eec4d9-2afa9b824f72748b_1000057231.jpg
f4075a96-8cf891546d9caeff_1000057534.jpg
bfff1e13-4a20450a838047ef_IMG_20240107_161535_HDR.jpg
e192e64b-56f16cbb4ac468fa_IMG-20230720-WA0026.jpg
84feaee6-75f8a3e4f414aa32_1000057587.jpg
c828593d-e8eb24d9a9110084_1000057484.jpg
577cb0ba-8b89acaf11819231_1000057540.jpg


Load annotations without Images

In [12]:
import os

# Get the value of the environment variable
labelstudio_token = os.getenv('LABELSTUDIO_TOKEN')
annotations_dir = "annotations/"
filename = "single_annotations"
file_path = os.path.join(annotations_dir, filename + ".csv")
project_id_single_groceries=4
project_id_mixed_groceries=1
!echo {file_path} 
!curl http://foodsnapai.com:8080/api/projects/1/export?exportType=CSV -H 'Authorization: Token {labelstudio_token}' --output {file_path} 

annotations/single_annotations.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4168k  100 4168k    0     0   341k      0  0:00:12  0:00:12 --:--:-- 1043k


Convert annotations and images and preprocess them

In [13]:
import pandas as pd
import os

%run 'convert_csv_to_mlflow_csv.py' {file_path} 

mlflow_filename = os.path.join(annotations_dir, filename + "_mlflow.csv") 
# Read the CSV file into a pandas DataFrame
df = pd.read_csv(mlflow_filename)

# Count distinct values in the 'path' column
distinct_files = df['path'].unique()
num_distinct_files = len(distinct_files)

# Rename the file to include the number of images
base_name, extension = os.path.splitext(filename)
new_filename =os.path.join(annotations_dir, f"{filename}_{num_distinct_files}_mlflow.csv") 
os.rename(mlflow_filename, new_filename)

print(f"Distinct files in dataset: {num_distinct_files}")
print(f"File renamed to: {new_filename}")
annotations_full_filename = new_filename

Distinct files in dataset: 2709
File renamed to: annotations/single_annotations_2709_mlflow.csv


In [14]:
# merge classes here
# curret merge string: not:Tomato,Cucumber,Bell-Pepper,Onion,Carrot,Banana,Egg,Scallion,Lemon,Potato,Zucchini,Garlic,Apple,Pumpkin,Ginger
# not:Tomato,Cucumber,Bell-Pepper,Onion,Carrot,Banana,Egg,Scallion,Lemon,Potato,Zucchini,Garlic,Apple,Pumpkin,Ginger,Lime,Avocado,Mango,Broccoli,Chilli,Cauliflower,Eggplant,Cabagge,Orange
# annotations_full_filename = "annotations_single"    +"_mlflow.csv"
%run 'convert_pascal_to_googlecsv.py' --merged_csv {annotations_full_filename} 

Skipping automatic class merge.
['Tomato', 'Cucumber', 'Bell-Pepper', 'Onion', 'Carrot', 'Banana', 'Egg', 'Scallion', 'Lemon', 'Potato', 'Zucchini', 'Garlic', 'Apple', 'Pumpkin', 'Ginger', 'Lime', 'Avocado', 'Mango', 'Broccoli', 'Chilli', 'Cauliflower', 'Eggplant', 'Cabagge', 'Orange']
Merging classes ['Canned-Tomato', 'Cheese', 'Lentils', 'Curd', 'Yoghurt', 'Beans', 'Peas', 'Leek', 'Salad', 'Strawberry', 'Skyr', 'Oil', 'Plantmilk', 'Spinach', 'Mushroom', 'Tofu', 'Rice', 'Corn', 'Vinegar', 'Soysauce', 'Balsamico', 'Butter', 'Chickpeas', 'Nuts', 'Pasta', 'Milk', 'Flour'] to 'other'
Train-test-validation split applied to annotations/single_annotations_2709_mlflow_shuffled.csv


Now rename the file paths in the file to their actual location (find & replace)

In [15]:
import csv
import os

# Constant path to prepend
constant_path = '/home/alex/allImgs_extracted_smaller/'

# CSV file path
csv_dir, csv_filename = os.path.split(annotations_full_filename)
csv_base, csv_ext = os.path.splitext(csv_filename)
# add shuffled suffix because convert_pascal_to_googlecsv.py appends this
csv_file_path = os.path.join(csv_dir, f"{csv_base}_shuffled{csv_ext}")

# Open the CSV file and modify the paths in the second column
with open(csv_file_path, 'r+', newline='') as file:
    csv_reader = csv.reader(file)
    rows = list(csv_reader)

    for row in rows:
        if len(row) > 1:  # Ensure the row has at least two columns
            # Assuming the file path is in the second column (index 1)
            img_file_path = row[1]
            img_file_name = os.path.basename(img_file_path)  # Extract the filename from the original path
            row[1] = os.path.join(constant_path, img_file_name)  # Create the new path with the constant path and filename

    # Move the file pointer to the beginning
    file.seek(0)

    # Write the modified rows back to the CSV file
    csv_writer = csv.writer(file)
    csv_writer.writerows(rows)

print(f"CSV file '{csv_file_path}' modified with constant path replacing each file path in the second column.")



CSV file 'annotations/single_annotations_2709_mlflow_shuffled.csv' modified with constant path replacing each file path in the second column.


In [16]:
!pwd
%run 'png_to_jpeg.py' {csv_file_path} 
#%run 'png_to_jpeg.py' annotations1650_mlflow_shuffled_n.csv execute this line after training session 

/home/alex/tflite_model_maker_wsl2
convert png image files to jpeg imgs


0it [00:00, ?it/s]

Converted 0
Ignored 17220 files because: Not a PNG file
Ignored 1547 files because: Not found


In [23]:
# make images smaller (run only when new images have been loaded)

%run preproc_imgs.py /home/alex/allImgs_extracted /home/alex/allImgs_extracted_smaller 800
#%run preproc_imgs.py /mnt/z/annotated_individual/images/ /home/alex/individual_tiny_extracted_smaller 800

Resizing images:  15%|█▍        | 1069/7266 [03:33<15:35,  6.62it/s]

Error processing e82677308afcf9cb_IMG_20230622_114747.jpg: cannot identify image file '/home/alex/allImgs_extracted/e82677308afcf9cb_IMG_20230622_114747.jpg'


Resizing images:  45%|████▍     | 3251/7266 [10:55<10:08,  6.59it/s]

Error processing 2ea95cbde7c6aca9_IMG_20230622_115029.jpg: cannot identify image file '/home/alex/allImgs_extracted/2ea95cbde7c6aca9_IMG_20230622_115029.jpg'


Resizing images: 100%|██████████| 7266/7266 [24:53<00:00,  4.87it/s]

7 images did not need resizing. Copied to /home/alex/allImgs_extracted_smaller/4059e4cdc4670770_20231012_131640.jpg
7257 did need resizing. Copied to /home/alex/allImgs_extracted_smaller/4059e4cdc4670770_20231012_131640.jpg


In [16]:
import os

dir, file_name = os.path.split(csv_file_path)
file_base, ext = os.path.splitext(file_name)

normalized_csv = os.path.join(dir, f"{file_base}_n{ext}")

%run normalize_csv.py {csv_file_path}  {normalized_csv} 

check the stats of the dataset splits

In [17]:
#annotations_base_filename = "annotations1900"
%run 'get_label_stats.py' {normalized_csv} 

There are 2709 different files in this set
There are 25 different classes in this set
Classes: Onion, Lemon, Ginger, Orange, Eggplant, Scallion, Cucumber, Lime, Banana, Tomato, Apple, Carrot, other, Zucchini, Broccoli, Avocado, Bell-Pepper, Chilli, Egg, Pumpkin, Cabagge, Garlic, Potato, Mango, Cauliflower
Onion occurs in 827 different files
Lemon occurs in 554 different files
Ginger occurs in 447 different files
Orange occurs in 286 different files
Eggplant occurs in 182 different files
Scallion occurs in 564 different files
Cucumber occurs in 688 different files
Lime occurs in 326 different files
Banana occurs in 579 different files
Tomato occurs in 798 different files
Apple occurs in 396 different files
Carrot occurs in 555 different files
other occurs in 1412 different files
Zucchini occurs in 481 different files
Broccoli occurs in 122 different files
Avocado occurs in 301 different files
Bell-Pepper occurs in 650 different files
Chilli occurs in 247 different files
Egg occurs in 33

### Optionally remove other class

In [7]:
import os

def remove_lines_with_infix(input_filename, infix_to_remove):
    directory, file_name = os.path.split(input_filename)
    file_base, file_extension = os.path.splitext(file_name)
    output_filename = os.path.join(directory, file_base + "_no_" + infix_to_remove + file_extension)

    with open(input_filename, 'r') as file:
        lines = file.readlines()

    # Remove lines containing the infix
    filtered_lines = [line for line in lines if infix_to_remove not in line]

    with open(output_filename, 'w') as file:
        file.writelines(filtered_lines)

    print(f"Lines containing '{infix_to_remove}' removed. Output written to '{output_filename}'.")

# Example usage
# remove_lines_with_infix('/path/to/your/file.txt', 'infix_to_remove')


remove_lines_with_infix(normalized_csv,"other")    

Lines containing 'other' removed. Output written to 'annotations/annotations_2709_mlflow_shuffled_n_no_other.csv'.


Now you can start to load the model and train!

In [10]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

# The current code uses Focal loss which has already weighted loss because of alpha and gamma
model_name = 'efficientdet-lite0' # EfficientDetLite1Spec must also be set accordingly!
custom_model_dir_name = 'model_'+"2709_test"#str(num_distinct_files)
epochs = 50
batch_size = 8
model_dir = 'models/'+model_name+'/'+custom_model_dir_name+'_e'+str(epochs)+'_b'+str(batch_size)
#spec = model_spec.get('efficientdet_lite1')
# check this url to check valid hparam values
# https://github.com/tensorflow/examples/blob/master/tensorflow_examples/lite/model_maker/third_party/efficientdet/hparams_config.py
spec = object_detector.EfficientDetLite0Spec( # change this also to correct model spec
    model_name = model_name,
    model_dir='/home/alex/checkpoints/',
    hparams='grad_checkpoint=true,strategy=gpus',
    epochs=epochs, batch_size=batch_size,
    steps_per_execution=1, moving_average_decay=0,
    var_freeze_expr='(efficientnet|fpn_cells|resample_p6)',
    tflite_max_detections=25
)


# Load the dataset

If you get an error here run the png to jpeg script again. Inference seems to fuck it up ^^

In [9]:
#annotations_base_filename = 'annotations1900'
# png to jpeg needs to run again after inference
# %run 'png_to_jpeg.py' {normalized_csv} 
import os
current_directory = os.getcwd()
print("Current Working Directory:", current_directory)

# file_path = '/home/alex/tflite_model_maker_wsl2/'+annotations_base_filename +"_mlflow_shuffled_n.csv"
file_path = normalized_csv

if os.path.exists(file_path):
    print(f"The file '{file_path}' exists.")
train_data, validation_data, test_data = object_detector.DataLoader.from_csv(file_path)


Current Working Directory: /home/alex/tflite_model_maker_wsl2
The file 'annotations/annotations_2709_mlflow_shuffled_n.csv' exists.


# Train the model

In [11]:
model = object_detector.create(train_data, model_spec=spec, train_whole_model=True, validation_data=validation_data)

Epoch 1/50


2024-01-30 12:49:55.991401: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101


270/270 [==============================] - 151s 486ms/step - det_loss: 1.6713 - cls_loss: 1.0918 - box_loss: 0.0116 - reg_l2_loss: 0.0671 - loss: 1.7384 - learning_rate: 0.0090 - gradient_norm: 0.9606 - val_det_loss: 1.5957 - val_cls_loss: 1.0479 - val_box_loss: 0.0110 - val_reg_l2_loss: 0.0671 - val_loss: 1.6627
Epoch 2/50
270/270 [==============================] - 125s 465ms/step - det_loss: 1.4725 - cls_loss: 0.9688 - box_loss: 0.0101 - reg_l2_loss: 0.0670 - loss: 1.5396 - learning_rate: 0.0100 - gradient_norm: 1.3262 - val_det_loss: 1.5093 - val_cls_loss: 0.9836 - val_box_loss: 0.0105 - val_reg_l2_loss: 0.0670 - val_loss: 1.5764
Epoch 3/50
 97/270 [=========>....................] - ETA: 1:27 - det_loss: 1.4225 - cls_loss: 0.9336 - box_loss: 0.0098 - reg_l2_loss: 0.0670 - loss: 1.4895 - learning_rate: 0.0100 - gradient_norm: 1.4896

KeyboardInterrupt: 

### Write the model maker class list to a file!

In [25]:
# try out this instead maybe dl new version first (did not work with old one) model.export_labels(model_dir+'/label_map.json')

import json
import os

if not os.path.exists(model_dir):
    os.makedirs(model_dir)

label_map = model.model_spec.config.label_map.as_dict()
# Writing the dictionary to a JSON file
with open(model_dir+'/label_map.json', 'w') as file:
    json.dump(label_map, file)

# run tensorboard to monitor

In [57]:
# !tensorboard --logdir=/home/alex/checkpoints_lite1_1700imgs/ --port=6006

Run on test data

In [26]:
model.evaluate(test_data)

31/31 [==============================] - 8s 141ms/step



{'AP': 0.08548653,
 'AP50': 0.1361972,
 'AP75': 0.09609742,
 'APs': 0.0,
 'APm': 0.055434335,
 'APl': 0.10378562,
 'ARmax1': 0.09282439,
 'ARmax10': 0.16570528,
 'ARmax100': 0.17741416,
 'ARs': 0.0,
 'ARm': 0.11349891,
 'ARl': 0.20717403,
 'AP_/other': 0.2532248,
 'AP_/Potato': 0.12074429,
 'AP_/Tomato': 0.15464021,
 'AP_/Onion': 0.12620524,
 'AP_/Apple': 0.09803807,
 'AP_/Banana': 0.2172316,
 'AP_/Pumpkin': 0.0325109,
 'AP_/Scallion': 0.061618336,
 'AP_/Avocado': 0.09182611,
 'AP_/Lemon': 0.08666255,
 'AP_/Bell-Pepper': 0.14741343,
 'AP_/Carrot': 0.049910072,
 'AP_/Egg': 0.13089462,
 'AP_/Cucumber': 0.07713252,
 'AP_/Zucchini': 0.044840463,
 'AP_/Chilli': 0.0001142422,
 'AP_/Lime': 0.03753449,
 'AP_/Ginger': 0.08042903,
 'AP_/Garlic': 0.041250616,
 'AP_/Cabagge': 0.07485298,
 'AP_/Mango': 0.0,
 'AP_/Eggplant': 0.027572501,
 'AP_/Broccoli': 0.0970297,
 'AP_/Cauliflower': 0.0}

### Export the model to tflite model

In [27]:
model.export(export_dir=model_dir)
print(f"exported to model to {model_dir}")

2024-01-07 22:55:52.942000: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2024-01-07 22:56:07.102552: W tensorflow/core/common_runtime/graph_constructor.cc:803] Node 'resample_p7/PartitionedCall' has 1 outputs but the _output_shapes attribute specifies shapes for 3 outputs. Output shapes may be inaccurate.


Estimated count of arithmetic ops: 1.834 G  ops, equivalently 0.917 G  MACs


2024-01-07 22:56:11.098355: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2024-01-07 22:56:11.098387: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2024-01-07 22:56:11.101489: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /tmp/tmpe1doj34a
2024-01-07 22:56:11.160670: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2024-01-07 22:56:11.160691: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /tmp/tmpe1doj34a
2024-01-07 22:56:11.366282: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2024-01-07 22:56:12.367672: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: /tmp/tmpe1doj34a
2024-01-07 22:56:12.809680: I tensorflow/cc/saved_model/loader.cc:301] SavedModel load for tags { serve }; Status: success: OK. Took 1708856

Estimated count of arithmetic ops: 1.834 G  ops, equivalently 0.917 G  MACs
exported to model to models/efficientdet-lite0/model_2488_more_classes_e50_b8


Evaluate on TFLITE

In [9]:
# TEST
#model_dir='models/efficientdet-lite0/model_1907_e50_b16/model.tflite'
model.evaluate_tflite(model_dir+'/model.tflite', test_data)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


190/190 [==============================] - 254s 1s/step



{'AP': 0.072127916,
 'AP50': 0.112064324,
 'AP75': 0.07951266,
 'APs': 0.0,
 'APm': 0.034183107,
 'APl': 0.08941372,
 'ARmax1': 0.06421327,
 'ARmax10': 0.10355284,
 'ARmax100': 0.105854,
 'ARs': 0.0,
 'ARm': 0.054981574,
 'ARl': 0.12072811,
 'AP_/Cabagge': 0.058613863,
 'AP_/Cauliflower': 0.0,
 'AP_/Lime': 0.025742574,
 'AP_/other': 0.22228026,
 'AP_/Chilli': 0.000990099,
 'AP_/Tomato': 0.065596685,
 'AP_/Cucumber': 0.01336198,
 'AP_/Mango': 0.0,
 'AP_/Lemon': 0.059609786,
 'AP_/Bell-Pepper': 0.10160959,
 'AP_/Carrot': 0.022990871,
 'AP_/Onion': 0.12955928,
 'AP_/Zucchini': 0.0016142919,
 'AP_/Ginger': 0.15429042,
 'AP_/Garlic': 0.062772274,
 'AP_/Potato': 0.19233683,
 'AP_/Scallion': 0.0071628676,
 'AP_/Avocado': 0.0,
 'AP_/Broccoli': 0.054455444,
 'AP_/Pumpkin': 0.14257425,
 'AP_/Banana': 0.15792875,
 'AP_/Eggplant': 0.0,
 'AP_/Egg': 0.20838435,
 'AP_/Apple': 0.049195543}

Run inference script for visual output

In [10]:
# needs to be after training for some reason

%run 'png_to_jpeg.py' {annotations_base_filename +"_mlflow_shuffled_n.csv"}


%run do_inference.py --input_csv {annotations_base_filename +"_mlflow_shuffled_n.csv"}\
                       --model_url {model_dir}/model.tflite --output_dir output_inference12 --label_file {model_dir}/label_map.json

convert png image files to jpeg imgs


0it [00:00, ?it/s]

Converted 0
Ignored 16196 files because: Not a PNG file
Ignored 0 files because: Not found
Predications will be saved to output_inference12


100%|██████████| 190/190 [04:34<00:00,  1.45s/it]
